In [1]:
import psycopg2 as pg2
import pandas as pd

con = pg2.connect(host='localhost', user='postgres', database='postgres', password='passpostgres')
con.autocommit = True
cur = con.cursor()

def select(sql):
    return pd.read_sql(sql,con)

In [2]:
sql = '''
drop table if exists sales_month;
create table sales_month as

select a.* , b.invoice_count, b.price_sum, 1 as customer,
case when a.reg_month = a.month then 1 else 0 end as new,
case when b.invoice_count > 0 then 1 else 0 end as active,

max(case when b.invoice_count > 0 then a.month else null end) over (partition by a.customerid order by a.month) as last_transaction,

extract(year from age(a.month, max(case when b.invoice_count > 0 then a.month else null end) over (partition by a.customerid order by a.month))) * 12 +
extract(month from age(a.month, max(case when b.invoice_count > 0 then a.month else null end) over (partition by a.customerid order by a.month)))
as last_order_count,

extract(year from age(a.month, a.reg_month)) * 12 +
extract(month from age(a.month, a.reg_month))
as lifetime

from user_month_1 a
left join sales_month_agg b
on a.customerid = b.customerid and a.month = b.month

'''

In [3]:
cur.execute(sql)

In [4]:
sql = '''
select a.*,
max(b.month),

extract(year from age(a.month, max(b.month))) * 12 +
extract(month from age(a.month, max(b.month)))
as last_order_month,

sum(b.invoice_count) as invoice_sum,
sum(b.price_sum) as price_sum

from

(select a.customerid, a.month, case when a.invoice_count > 0 then 1 else 0 end as target
from sales_month a
where a.month = '2011-10-01' and a.lifetime > 0) a

left join sales_month_agg b
on a.customerid = b.customerid and a.month > b.month

group by a.customerid, a.month, a.target
'''

In [5]:
df = select(sql)

In [6]:
df

,customerid,month,target,max,last_order_month,invoice_sum,price_sum
0,12346.0,2011-10-01,0,2011-01-01,9.0,2.0,0.00
1,12347.0,2011-10-01,1,2011-08-01,2.0,5.0,2790.86
2,12348.0,2011-10-01,0,2011-09-01,1.0,4.0,1797.24
3,12350.0,2011-10-01,0,2011-02-01,8.0,1.0,334.40
4,12352.0,2011-10-01,0,2011-09-01,1.0,10.0,1233.68
...,...,...,...,...,...,...,...
3653,18280.0,2011-10-01,0,2011-03-01,7.0,1.0,180.60
3654,18281.0,2011-10-01,0,2011-06-01,4.0,1.0,80.82
3655,18282.0,2011-10-01,0,2011-08-01,2.0,2.0,98.76
3656,18283.0,2011-10-01,1,2011-09-01,1.0,10.0,1120.67


In [7]:
df['recency'] = df['last_order_month'].rank(pct=True,ascending=False,method='max')

In [8]:
df

,customerid,month,target,max,last_order_month,invoice_sum,price_sum,recency
0,12346.0,2011-10-01,0,2011-01-01,9.0,2.0,0.00,0.088026
1,12347.0,2011-10-01,1,2011-08-01,2.0,5.0,2790.86,0.644068
2,12348.0,2011-10-01,0,2011-09-01,1.0,4.0,1797.24,1.000000
3,12350.0,2011-10-01,0,2011-02-01,8.0,1.0,334.40,0.126845
4,12352.0,2011-10-01,0,2011-09-01,1.0,10.0,1233.68,1.000000
...,...,...,...,...,...,...,...,...
3653,18280.0,2011-10-01,0,2011-03-01,7.0,1.0,180.60,0.186987
3654,18281.0,2011-10-01,0,2011-06-01,4.0,1.0,80.82,0.399672
3655,18282.0,2011-10-01,0,2011-08-01,2.0,2.0,98.76,0.644068
3656,18283.0,2011-10-01,1,2011-09-01,1.0,10.0,1120.67,1.000000


In [9]:
df['frequency'] = df['invoice_sum'].rank(pct=True,method='max')

In [10]:
df['monetary'] = df['price_sum'].rank(pct=True,method='max')

In [11]:
df

,customerid,month,target,max,last_order_month,invoice_sum,price_sum,recency,frequency,monetary
0,12346.0,2011-10-01,0,2011-01-01,9.0,2.0,0.00,0.088026,0.536359,0.014762
1,12347.0,2011-10-01,1,2011-08-01,2.0,5.0,2790.86,0.644068,0.793876,0.894751
2,12348.0,2011-10-01,0,2011-09-01,1.0,4.0,1797.24,1.000000,0.739475,0.812739
3,12350.0,2011-10-01,0,2011-02-01,8.0,1.0,334.40,0.126845,0.345817,0.335429
4,12352.0,2011-10-01,0,2011-09-01,1.0,10.0,1233.68,1.000000,0.920995,0.721159
...,...,...,...,...,...,...,...,...,...,...
3653,18280.0,2011-10-01,0,2011-03-01,7.0,1.0,180.60,0.186987,0.345817,0.159377
3654,18281.0,2011-10-01,0,2011-06-01,4.0,1.0,80.82,0.399672,0.345817,0.043466
3655,18282.0,2011-10-01,0,2011-08-01,2.0,2.0,98.76,0.644068,0.536359,0.054948
3656,18283.0,2011-10-01,1,2011-09-01,1.0,10.0,1120.67,1.000000,0.920995,0.700656


In [12]:
df['score'] = df['recency'] * .5 + df['frequency'] *.2 + df['monetary'] * .3

In [13]:
df

,customerid,month,target,max,last_order_month,invoice_sum,price_sum,recency,frequency,monetary,score
0,12346.0,2011-10-01,0,2011-01-01,9.0,2.0,0.00,0.088026,0.536359,0.014762,0.155714
1,12347.0,2011-10-01,1,2011-08-01,2.0,5.0,2790.86,0.644068,0.793876,0.894751,0.749235
2,12348.0,2011-10-01,0,2011-09-01,1.0,4.0,1797.24,1.000000,0.739475,0.812739,0.891717
3,12350.0,2011-10-01,0,2011-02-01,8.0,1.0,334.40,0.126845,0.345817,0.335429,0.233215
4,12352.0,2011-10-01,0,2011-09-01,1.0,10.0,1233.68,1.000000,0.920995,0.721159,0.900547
...,...,...,...,...,...,...,...,...,...,...,...
3653,18280.0,2011-10-01,0,2011-03-01,7.0,1.0,180.60,0.186987,0.345817,0.159377,0.210470
3654,18281.0,2011-10-01,0,2011-06-01,4.0,1.0,80.82,0.399672,0.345817,0.043466,0.282039
3655,18282.0,2011-10-01,0,2011-08-01,2.0,2.0,98.76,0.644068,0.536359,0.054948,0.445790
3656,18283.0,2011-10-01,1,2011-09-01,1.0,10.0,1120.67,1.000000,0.920995,0.700656,0.894396


In [14]:
df['target'].mean()

0.29332968835429196

In [15]:
df[df['score'] >= .7]['target'].mean()

0.47123719464145

In [16]:
df['score_pcntl'] = df['score'].rank(pct=True,ascending=False,method='max')

In [17]:
import numpy as np

In [18]:
np.linspace(0,1,101)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])

In [19]:
df['score_cut'] = pd.cut(df['score_pcntl'],np.linspace(0,1,101))

In [20]:
df

,customerid,month,target,max,last_order_month,invoice_sum,price_sum,recency,frequency,monetary,score,score_pcntl,score_cut
0,12346.0,2011-10-01,0,2011-01-01,9.0,2.0,0.00,0.088026,0.536359,0.014762,0.155714,0.956534,"(0.95, 0.96]"
1,12347.0,2011-10-01,1,2011-08-01,2.0,5.0,2790.86,0.644068,0.793876,0.894751,0.749235,0.291689,"(0.29, 0.3]"
2,12348.0,2011-10-01,0,2011-09-01,1.0,4.0,1797.24,1.000000,0.739475,0.812739,0.891717,0.143794,"(0.14, 0.15]"
3,12350.0,2011-10-01,0,2011-02-01,8.0,1.0,334.40,0.126845,0.345817,0.335429,0.233215,0.872881,"(0.87, 0.88]"
4,12352.0,2011-10-01,0,2011-09-01,1.0,10.0,1233.68,1.000000,0.920995,0.721159,0.900547,0.133953,"(0.13, 0.14]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3653,18280.0,2011-10-01,0,2011-03-01,7.0,1.0,180.60,0.186987,0.345817,0.159377,0.210470,0.898852,"(0.89, 0.9]"
3654,18281.0,2011-10-01,0,2011-06-01,4.0,1.0,80.82,0.399672,0.345817,0.043466,0.282039,0.815746,"(0.81, 0.82]"
3655,18282.0,2011-10-01,0,2011-08-01,2.0,2.0,98.76,0.644068,0.536359,0.054948,0.445790,0.626572,"(0.62, 0.63]"
3656,18283.0,2011-10-01,1,2011-09-01,1.0,10.0,1120.67,1.000000,0.920995,0.700656,0.894396,0.141607,"(0.14, 0.15]"


In [22]:
lift = df.groupby('score_cut')['target'].agg(['sum', 'count']).reset_index()

In [23]:
lift

,score_cut,sum,count
0,"(0.0, 0.01]",34,36
1,"(0.01, 0.02]",31,36
2,"(0.02, 0.03]",31,37
3,"(0.03, 0.04]",24,37
4,"(0.04, 0.05]",26,36
...,...,...,...
95,"(0.95, 0.96]",4,36
96,"(0.96, 0.97]",6,37
97,"(0.97, 0.98]",4,36
98,"(0.98, 0.99]",6,37


In [27]:
lift['target_cum'] = lift['sum'].cumsum()

In [32]:
lift['count_cum'] = lift['count'].cumsum()

In [33]:
lift['target_pct'] = lift['target_cum'] / lift['sum'].sum()

In [38]:
lift['score_right'] = lift['score_cut'].apply(lambda x: x.right).astype(float)

In [39]:
lift

,score_cut,sum,count,target_cum,target_pct,count_cum,score_right
0,"(0.0, 0.01]",34,36,34,0.031687,36,0.01
1,"(0.01, 0.02]",31,36,65,0.060578,72,0.02
2,"(0.02, 0.03]",31,37,96,0.089469,109,0.03
3,"(0.03, 0.04]",24,37,120,0.111836,146,0.04
4,"(0.04, 0.05]",26,36,146,0.136067,182,0.05
...,...,...,...,...,...,...,...
95,"(0.95, 0.96]",4,36,1055,0.983225,3511,0.96
96,"(0.96, 0.97]",6,37,1061,0.988816,3548,0.97
97,"(0.97, 0.98]",4,36,1065,0.992544,3584,0.98
98,"(0.98, 0.99]",6,37,1071,0.998136,3621,0.99


In [41]:
lift['uplift'] = lift['target_pct'] / lift['score_right']

In [42]:
lift[lift['score_cut'].isin([.05,.1,.15,.2,.3,.4,.5])]

,score_cut,sum,count,target_cum,target_pct,count_cum,score_right,uplift
4,"(0.04, 0.05]",26,36,146,0.136067,182,0.05,2.721342
9,"(0.09, 0.1]",13,36,240,0.223672,365,0.10,2.236719
14,"(0.14, 0.15]",18,36,314,0.292637,548,0.15,1.950916
19,"(0.19, 0.2]",16,36,381,0.355079,731,0.20,1.775396
29,"(0.29, 0.3]",18,37,532,0.495806,1097,0.30,1.652687
39,"(0.39, 0.4]",15,37,663,0.617894,1463,0.40,1.544734
49,"(0.49, 0.5]",5,37,776,0.723206,1829,0.50,1.446412
